In [3]:
import os
import pandas as pd
from pathlib import Path
import glob
from tqdm import tqdm

# Paths for MERRA2 2017 datasets
MERRA2_INPUT_PATH = r"Z:\Thesis\Data\GEE\MERRA2_aer\MERRA2_num_data\2017"
MERRA2_OUTPUT_PATH = r"Z:\Thesis\Data\GEE\MERRA2_aer\MERRA2_num_data\Monthly_stats\2017"

# Columns to calculate statistics for
AOD_COLUMNS = ["DUSMASS", "DUSMASS25", "DUCMASS", "DUCMASS25", "DUFLUXU", "DUFLUXV"]

def process_merra2_files(input_path, output_path):
    """Process MERRA2 parquet files for the year 2017 to calculate monthly statistics."""
    Path(output_path).mkdir(parents=True, exist_ok=True)

    # Find all parquet files in the specified input directory
    parquet_files = sorted(glob.glob(os.path.join(input_path, "**", "*.parquet"), recursive=True))

    all_data = []

    print("Reading parquet files and concatenating data...")
    for file in tqdm(parquet_files, desc="Processing files"):
        df = pd.read_parquet(file)

        # Ensure 'time' column is in datetime format
        if 'time' not in df.columns:
            raise KeyError(f"'time' column not found in file {file}")
        df['time'] = pd.to_datetime(df['time'])
        
        # Filter required columns
        columns_to_keep = ['time', 'lat', 'lon'] + AOD_COLUMNS
        df = df[columns_to_keep]
        
        all_data.append(df)

    # Combine all data into one DataFrame
    combined_df = pd.concat(all_data, ignore_index=True)

    print("Grouping data by month and calculating statistics...")
    # Group by month and calculate statistics for specified columns
    grouped = combined_df.groupby([pd.Grouper(key='time', freq='M'), 'lat', 'lon'])
    monthly_stats = grouped[AOD_COLUMNS].agg(['min', 'max', 'mean']).reset_index()

    # Flatten multi-level column index
    monthly_stats.columns = [
        f"{col[0]}_{col[1]}" if col[1] else col[0] for col in monthly_stats.columns
    ]

    # Save the output as a parquet file
    output_file = os.path.join(output_path, "MERRA2_2017_monthly_stats.parquet")
    monthly_stats.to_parquet(output_file, index=False)

    print(f"Saved MERRA2 monthly statistics to {output_file}")

if __name__ == "__main__":
    process_merra2_files(MERRA2_INPUT_PATH, MERRA2_OUTPUT_PATH)


Reading parquet files and concatenating data...


Processing files: 100%|█████████████████████████| 9100/9100 [06:53<00:00, 21.99it/s]


Grouping data by month and calculating statistics...


C:\Users\Charl\AppData\Local\Temp\ipykernel_28652\856486451.py:43: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  grouped = combined_df.groupby([pd.Grouper(key='time', freq='M'), 'lat', 'lon'])


Saved MERRA2 monthly statistics to Z:\Thesis\Data\GEE\MERRA2_aer\MERRA2_num_data\Monthly_stats\2017\MERRA2_2017_monthly_stats.parquet


In [1]:
import os
import pandas as pd
from pathlib import Path
import glob
from tqdm import tqdm

# Base paths for input and output
MERRA2_BASE_INPUT_PATH = r"Z:\Thesis\Data\GEE\MERRA2_aer\MERRA2_num_data"
MERRA2_BASE_OUTPUT_PATH = r"Z:\Thesis\Data\GEE\MERRA2_aer\MERRA2_num_data\Monthly_stats"

# Columns to calculate statistics for
AOD_COLUMNS = ["DUSMASS", "DUSMASS25", "DUCMASS", "DUCMASS25", "DUFLUXU", "DUFLUXV"]

def process_merra2_files_for_year(year, input_base_path, output_base_path):
    """
    Process MERRA2 parquet files for a specific year to calculate monthly statistics.
    """
    input_path = os.path.join(input_base_path, str(year))
    output_path = os.path.join(output_base_path, str(year))

    Path(output_path).mkdir(parents=True, exist_ok=True)

    # Find all parquet files in the specified input directory
    parquet_files = sorted(glob.glob(os.path.join(input_path, "**", "*.parquet"), recursive=True))

    if not parquet_files:
        print(f"No parquet files found for the year {year}. Skipping...")
        return

    all_data = []

    print(f"Processing files for the year {year}...")
    for file in tqdm(parquet_files, desc=f"Processing files for {year}"):
        df = pd.read_parquet(file)

        # Ensure 'time' column is in datetime format
        if 'time' not in df.columns:
            raise KeyError(f"'time' column not found in file {file}")
        df['time'] = pd.to_datetime(df['time'])
        
        # Filter required columns
        columns_to_keep = ['time', 'lat', 'lon'] + AOD_COLUMNS
        df = df[columns_to_keep]
        
        all_data.append(df)

    # Combine all data into one DataFrame
    combined_df = pd.concat(all_data, ignore_index=True)

    print(f"Grouping data by month and calculating statistics for the year {year}...")
    # Group by month and calculate statistics for specified columns
    grouped = combined_df.groupby([pd.Grouper(key='time', freq='ME'), 'lat', 'lon'])
    monthly_stats = grouped[AOD_COLUMNS].agg(['min', 'max', 'mean']).reset_index()

    # Flatten multi-level column index
    monthly_stats.columns = [
        f"{col[0]}_{col[1]}" if col[1] else col[0] for col in monthly_stats.columns
    ]

    # Save the output as a parquet file
    output_file = os.path.join(output_path, f"MERRA2_{year}_monthly_stats.parquet")
    monthly_stats.to_parquet(output_file, index=False)

    print(f"Saved MERRA2 monthly statistics for {year} to {output_file}")

def process_merra2_files_for_years(start_year, end_year, input_base_path, output_base_path):
    """
    Process MERRA2 parquet files for multiple years to calculate monthly statistics.
    """
    for year in range(start_year, end_year + 1):
        try:
            process_merra2_files_for_year(year, input_base_path, output_base_path)
        except Exception as e:
            print(f"An error occurred while processing the year {year}: {e}")

if __name__ == "__main__":
    START_YEAR = 1980
    END_YEAR = 2023
    process_merra2_files_for_years(START_YEAR, END_YEAR, MERRA2_BASE_INPUT_PATH, MERRA2_BASE_OUTPUT_PATH)


Processing files for the year 1980...


Processing files for 1980: 100%|████████████████| 9150/9150 [05:45<00:00, 26.48it/s]


Grouping data by month and calculating statistics for the year 1980...
Saved MERRA2 monthly statistics for 1980 to Z:\Thesis\Data\GEE\MERRA2_aer\MERRA2_num_data\Monthly_stats\1980\MERRA2_1980_monthly_stats.parquet
Processing files for the year 1981...


Processing files for 1981: 100%|████████████████| 9125/9125 [05:46<00:00, 26.37it/s]


Grouping data by month and calculating statistics for the year 1981...
Saved MERRA2 monthly statistics for 1981 to Z:\Thesis\Data\GEE\MERRA2_aer\MERRA2_num_data\Monthly_stats\1981\MERRA2_1981_monthly_stats.parquet
Processing files for the year 1982...


...

Grouping data by month and calculating statistics for the year 2021...
Saved MERRA2 monthly statistics for 2021 to Z:\Thesis\Data\GEE\MERRA2_aer\MERRA2_num_data\Monthly_stats\2021\MERRA2_2021_monthly_stats.parquet
Processing files for the year 2022...


Processing files for 2022: 100%|████████████████| 9100/9100 [06:02<00:00, 25.11it/s]


Grouping data by month and calculating statistics for the year 2022...
Saved MERRA2 monthly statistics for 2022 to Z:\Thesis\Data\GEE\MERRA2_aer\MERRA2_num_data\Monthly_stats\2022\MERRA2_2022_monthly_stats.parquet
Processing files for the year 2023...


Processing files for 2023: 100%|████████████████| 9100/9100 [06:08<00:00, 24.67it/s]


Grouping data by month and calculating statistics for the year 2023...
Saved MERRA2 monthly statistics for 2023 to Z:\Thesis\Data\GEE\MERRA2_aer\MERRA2_num_data\Monthly_stats\2023\MERRA2_2023_monthly_stats.parquet
